In [6]:
# Import des bibliothèques nécessaires
import numpy as np
import pandas as pd
import psycopg2
import psycopg2.extras
import os
import time
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import requests
import glob

# Charger les variables d'environnement
load_dotenv('../src/.env')

# Déclarer les variables nécessaires
data_folder_path = "C:\\Users\\USER\\Desktop\\ChatBot Rag\\Chatbot-RAG\\data"

# Initialiser le modèle d'embeddings local (gratuit, pas de quota!)
print("🔄 Chargement du modèle d'embeddings local...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # Modèle léger et performant (384 dimensions)
print("✅ Modèle d'embeddings chargé!")

# Configuration Groq (GRATUIT - ULTRA RAPIDE - GROS QUOTAS!)
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"
GROQ_MODEL = "llama-3.1-8b-instant"  # Ultra rapide: ~500 tokens/sec!
print(f"✅ Groq configuré avec {GROQ_MODEL}")

# Configuration de la connexion à la base de données
DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5432'),
    'dbname': os.getenv('DB_NAME', 'rag_chatbot'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD')
}

db_connection_str = f"dbname={DB_CONFIG['dbname']} user={DB_CONFIG['user']} password={DB_CONFIG['password']} host={DB_CONFIG['host']} port={DB_CONFIG['port']}"


def create_conversation_list_from_folder(folder_path: str) -> list[str]:
    """Charge et nettoie tous les fichiers .txt du dossier"""
    all_conversations = []
    encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']
    
    # Récupérer tous les fichiers .txt
    txt_files = glob.glob(os.path.join(folder_path, "*.txt"))
    print(f"📁 {len(txt_files)} fichiers .txt trouvés dans {folder_path}\n")
    
    for file_path in txt_files:
        file_name = os.path.basename(file_path)
        print(f"📄 Chargement de {file_name}...", end=" ")
        
        for encoding in encodings:
            try:
                with open(file_path, "r", encoding=encoding) as file:
                    text = file.read()
                    text_list = text.split("\n")
                    filtered_list = [chaine.removeprefix("     ") for chaine in text_list if chaine and not chaine.startswith("<")]
                    all_conversations.extend(filtered_list)
                    print(f"✅ {len(filtered_list)} conversations ({encoding})")
                    break
            except UnicodeDecodeError:
                continue
        else:
            print(f"❌ Impossible de décoder")
    
    print(f"\n🎉 Total: {len(all_conversations)} conversations chargées depuis {len(txt_files)} fichiers!")
    return all_conversations

def create_conversation_list(file_path: str) -> list[str]:
    """Charge et nettoie le fichier de conversation (version ancienne - un seul fichier)"""
    encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']
    
    for encoding in encodings:
        try:
            with open(file_path, "r", encoding=encoding) as file:
                text = file.read()
                text_list = text.split("\n")
                filtered_list = [chaine.removeprefix("     ") for chaine in text_list if chaine and not chaine.startswith("<")]
                print(f"✅ {len(filtered_list)} conversations chargées (encoding: {encoding})")
                return filtered_list
        except UnicodeDecodeError:
            continue
    
    raise ValueError(f"Impossible de décoder le fichier {file_path} avec les encodages testés: {encodings}")
    
def calculate_embeddings(corpus: str) -> list[float]:
    """Calcule les embeddings pour un texte donné avec Sentence Transformers (LOCAL - GRATUIT)"""
    # Plus besoin de délai - c'est local et gratuit!
    embedding = embedding_model.encode(corpus, convert_to_numpy=True)
    return embedding.tolist()

def save_embedding(corpus: str, embedding: list[float], cursor) -> None:
    """Sauvegarde le corpus et son embedding dans la base de données"""
    cursor.execute('''INSERT INTO embeddings (corpus, embedding) VALUES (%s, %s)''', (corpus, embedding)) 

def parse_postgres_array(array_string):
    """Parse PostgreSQL array string to list of floats - handles both FLOAT8[] and pgvector formats"""
    if isinstance(array_string, list):
        return array_string
    
    if isinstance(array_string, str):
        # Handle pgvector format: "[0.1,0.2,0.3]" 
        if array_string.startswith('[') and array_string.endswith(']'):
            cleaned = array_string.strip('[]')
            return [float(x) for x in cleaned.split(',')]
        
        # Handle FLOAT8[] format: "{0.1,0.2,0.3}"
        if array_string.startswith('{') and array_string.endswith('}'):
            cleaned = array_string.strip('{}')
            return [float(x) for x in cleaned.split(',')]
    
    return array_string
    
def cosine_distance(vec1: list[float], vec2: list[float]) -> float:
    """Calcule la distance cosinus entre deux vecteurs"""
    # Parse arrays if they are strings
    if isinstance(vec2, str):
        vec2 = parse_postgres_array(vec2)
    
    vec1_np = np.array(vec1, dtype=np.float64)
    vec2_np = np.array(vec2, dtype=np.float64)
    
    dot_product = np.dot(vec1_np, vec2_np)
    norm1 = np.linalg.norm(vec1_np)
    norm2 = np.linalg.norm(vec2_np)
    
    if norm1 == 0 or norm2 == 0:
        return 1.0
    
    similarity = dot_product / (norm1 * norm2)
    return 1 - similarity  # Distance = 1 - similarité
    
def similar_corpus(input_corpus: str, db_connection_str: str, top_k: int = 5) -> list[tuple]:
    """Trouve les corpus similaires dans la base de données"""
    # Calculer l'embedding de l'input avec Sentence Transformers (local)
    input_embedding = embedding_model.encode(input_corpus, convert_to_numpy=True).tolist()
    
    with psycopg2.connect(db_connection_str) as conn:
        with conn.cursor() as cur:
            # Récupérer tous les embeddings
            cur.execute("SELECT id, corpus, embedding FROM embeddings")
            all_results = cur.fetchall()
            
            # Calculer les distances en Python
            results_with_distance = []
            for id, corpus, embedding in all_results:
                distance = cosine_distance(input_embedding, embedding)
                results_with_distance.append((distance, id, corpus, embedding))
            
            # Trier par distance et prendre les top_k
            results_with_distance.sort(key=lambda x: x[0])
            top_results = [(id, corpus, embedding) for _, id, corpus, embedding in results_with_distance[:top_k]]
            
            return top_results

def generate_response(query: str, context: list[str]) -> str:
    """Génère une réponse avec Groq (GRATUIT - ULTRA RAPIDE!)"""
    
    prompt = f"""Tu es un assistant spécialisé dans l'analyse de conversations universitaires.
Réponds UNIQUEMENT avec les informations du contexte. Si l'info n'est pas dans le contexte, dis-le.

Contexte: {' '.join(context)}

Question: {query}

Réponse courte en français:"""
    
    try:
        start_time = time.time()
        
        response = requests.post(
            GROQ_URL,
            headers={
                "Authorization": f"Bearer {GROQ_API_KEY}",
                "Content-Type": "application/json"
            },
            json={
                "model": GROQ_MODEL,
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0.7,
                "max_tokens": 200  # Réponses courtes
            },
            timeout=10  # Groq est ultra rapide, 10s suffit
        )
        response.raise_for_status()
        
        result = response.json()
        elapsed = time.time() - start_time
        
        answer = result['choices'][0]['message']['content']
        print(f"⚡ Temps de réponse: {elapsed:.2f}s")
        
        return answer
        
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 429:
            return "❌ Quota Groq atteint. Attendez 1 minute."
        return f"❌ Erreur HTTP: {e.response.status_code}"
    except Exception as e:
        return f"❌ Erreur: {str(e)}"


🔄 Chargement du modèle d'embeddings local...
✅ Modèle d'embeddings chargé!
✅ Groq configuré avec llama-3.1-8b-instant
✅ Modèle d'embeddings chargé!
✅ Groq configuré avec llama-3.1-8b-instant


## Création de la table et insertion des données

In [7]:
# Connexion à la base de données et création de la table
with psycopg2.connect(db_connection_str) as conn:
    conn.autocommit = True
    with conn.cursor() as cur:
        # Supprimer la table si elle existe
        cur.execute("""DROP TABLE IF EXISTS embeddings""")
        print("🗑️  Table 'embeddings' supprimée (si existante)")
        
        # Créer l'extension pgvector (optionnel)
        try:
            cur.execute("""CREATE EXTENSION IF NOT EXISTS vector""")
            print("✅ Extension pgvector activée")
            use_pgvector = True
        except:
            print("⚠️  pgvector non disponible, utilisation de FLOAT8[]")
            use_pgvector = False
        
        # Créer la table - all-MiniLM-L6-v2 produit des embeddings de dimension 384
        if use_pgvector:
            cur.execute("""CREATE TABLE IF NOT EXISTS embeddings (
                        id SERIAL PRIMARY KEY, 
                        corpus TEXT,
                        embedding VECTOR(384),
                        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                        )""")
        else:
            cur.execute("""CREATE TABLE IF NOT EXISTS embeddings (
                        id SERIAL PRIMARY KEY, 
                        corpus TEXT,
                        embedding FLOAT8[],
                        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                        )""")
        
        print("✅ Table 'embeddings' créée avec succès!")
        
        # Charger et insérer les conversations depuis TOUS les fichiers .txt
        try:
            print("\n" + "="*80)
            print("🚀 CHARGEMENT DE TOUS LES FICHIERS .TXT")
            print("="*80 + "\n")
            
            corpus_list = create_conversation_list_from_folder(folder_path=data_folder_path)
            
            print("\n" + "="*80)
            print("💾 INSERTION DES EMBEDDINGS DANS LA BASE DE DONNÉES")
            print("="*80 + "\n")
       
            for i, corpus in enumerate(corpus_list, 1):
                try:
                    embedding = calculate_embeddings(corpus=corpus) 
                    save_embedding(corpus=corpus, embedding=embedding, cursor=cur)
                    
                    # Afficher la progression tous les 10 éléments
                    if i % 10 == 0 or i == len(corpus_list):
                        print(f"✅ [{i}/{len(corpus_list)}] Corpus insérés ({(i/len(corpus_list)*100):.1f}%)")
                        
                except Exception as e:
                    print(f"❌ Erreur lors de l'insertion du corpus {i}: {e}")
                
            conn.commit()
            print("\n" + "="*80)
            print(f"🎉 {len(corpus_list)} EMBEDDINGS SAUVEGARDÉS AVEC SUCCÈS!")
            print("="*80)
            
        except FileNotFoundError:
            print(f"⚠️  Dossier {data_folder_path} non trouvé")
            print("💡 Insertion de données de test à la place...")
            
            # Données de test
            test_data = [
                "Ceci est un exemple de texte pour le RAG",
                "PostgreSQL est un système de base de données relationnel",
                "Le chatbot utilise des embeddings pour la recherche sémantique"
            ]
            
            for corpus in test_data:
                try:
                    embedding = calculate_embeddings(corpus=corpus)
                    save_embedding(corpus=corpus, embedding=embedding, cursor=cur)
                except Exception as e:
                    print(f"❌ Erreur test data: {e}")
            
            conn.commit()
            print(f"✅ {len(test_data)} embeddings de test insérés!")


🗑️  Table 'embeddings' supprimée (si existante)
✅ Extension pgvector activée
✅ Table 'embeddings' créée avec succès!

🚀 CHARGEMENT DE TOUS LES FICHIERS .TXT

📁 41 fichiers .txt trouvés dans C:\Users\USER\Desktop\ChatBot Rag\Chatbot-RAG\data

📄 Chargement de 017_00000012.txt... ✅ 43 conversations (latin-1)
📄 Chargement de 018_00000013.txt... ✅ 9 conversations (latin-1)
📄 Chargement de 019_00000014.txt... ✅ 12 conversations (latin-1)
📄 Chargement de 020_00000015.txt... ✅ 13 conversations (latin-1)
📄 Chargement de 022_00000017.txt... ✅ 23 conversations (latin-1)
📄 Chargement de 023_00000018.txt... ✅ 14 conversations (latin-1)
📄 Chargement de 024_00000019.txt... ✅ 104 conversations (latin-1)
📄 Chargement de 027_0000001c.txt... ✅ 35 conversations (latin-1)
📄 Chargement de 028_0000001d.txt... ✅ 104 conversations (latin-1)
📄 Chargement de 029_0000001e.txt... ✅ 8 conversations (latin-1)
📄 Chargement de 030_0000001f.txt... ✅ 6 conversations (latin-1)
📄 Chargement de 037_00000026.txt... ✅ 7 conv

## Requête de recherche sémantique

In [8]:
# Test de recherche sémantique et génération de réponse
query = "Comment valider mon inscription universitaire?"

print(f"🔍 Recherche pour: '{query}'\n")

# 1. Trouver les corpus similaires
results = similar_corpus(input_corpus=query, db_connection_str=db_connection_str, top_k=3)

print(f"📊 Top {len(results)} résultats similaires:\n")
context = []
for i, (id, corpus, embedding) in enumerate(results, 1):
    print(f"{i}. [ID: {id}]")
    print(f"   Texte: {corpus}")
    print("-" * 80)
    context.append(corpus)

# 2. Générer une réponse avec le LLM
print("\n🤖 Génération de la réponse...\n")
response = generate_response(query=query, context=context)

print("="*80)
print(f"RÉPONSE GÉNÉRÉE :\n{response}")
print("="*80)


🔍 Recherche pour: 'Comment valider mon inscription universitaire?'

📊 Top 3 résultats similaires:

1. [ID: 1047]
   Texte: h: inscription administrative oui
--------------------------------------------------------------------------------
2. [ID: 1046]
   Texte: c: des inscriptions voilà oui
--------------------------------------------------------------------------------
3. [ID: 447]
   Texte: h: voilà c'est le Service Universitaire de l'Information et de l'Orientation parce que ici vous êtes à l'accueil de la fac de droit donc e
--------------------------------------------------------------------------------

🤖 Génération de la réponse...

📊 Top 3 résultats similaires:

1. [ID: 1047]
   Texte: h: inscription administrative oui
--------------------------------------------------------------------------------
2. [ID: 1046]
   Texte: c: des inscriptions voilà oui
--------------------------------------------------------------------------------
3. [ID: 447]
   Texte: h: voilà c'est le Servic

## Test avec différentes requêtes

In [9]:
# Tester différentes questions pertinentes aux conversations d'inscription universitaire
questions = [
    "Comment valider mon inscription?",
    "Que faire si je ne vois pas ma validation d'inscription?",
    "Quelle est la procédure pour confirmer mon bac?",
    "Comment contacter le service d'inscription?",
    "Quels sont les délais pour la pré-inscription?"
]

for i, question in enumerate(questions, 1):
    print(f"\n{'='*80}")
    print(f"QUESTION {i}: {question}")
    print('='*80)
    
    # Recherche sémantique
    results = similar_corpus(input_corpus=question, db_connection_str=db_connection_str, top_k=3)
    
    # Extraire le contexte
    context = [corpus for _, corpus, _ in results]
    
    # Générer la réponse avec OpenRouter
    try:
        response = generate_response(query=question, context=context)
        print(f"\n🤖 RÉPONSE:\n{response}")
    except Exception as e:
        print(f"❌ Erreur: {e}")
    
    print("\n📚 Sources utilisées:")
    for j, (id, corpus, _) in enumerate(results, 1):
        print(f"  {j}. [ID: {id}] {corpus[:80]}...")
    
    print(f"\n⏳ Pause de 3 secondes avant la prochaine requête...")
    time.sleep(3)  # Pause pour éviter les rate limits

print("\n" + "="*80)
print("✅ Test terminé!")



QUESTION 1: Comment valider mon inscription?
⚡ Temps de réponse: 0.49s

🤖 RÉPONSE:
Je ne dispose pas d'informations suffisantes pour répondre à votre question. Vous avez mentionné un souci avec une inscription, mais je ne sais pas de quelle manière. Voulez-vous plus d'informations ou peut-être expliquer votre problème ?

📚 Sources utilisées:
  1. [ID: 1047] h: inscription administrative oui...
  2. [ID: 1046] c: des inscriptions voilà oui...
  3. [ID: 258] h: un souci avec une inscription...

⏳ Pause de 3 secondes avant la prochaine requête...
⚡ Temps de réponse: 0.49s

🤖 RÉPONSE:
Je ne dispose pas d'informations suffisantes pour répondre à votre question. Vous avez mentionné un souci avec une inscription, mais je ne sais pas de quelle manière. Voulez-vous plus d'informations ou peut-être expliquer votre problème ?

📚 Sources utilisées:
  1. [ID: 1047] h: inscription administrative oui...
  2. [ID: 1046] c: des inscriptions voilà oui...
  3. [ID: 258] h: un souci avec une inscription.

In [10]:
# Vérifier toutes les données stockées
with psycopg2.connect(db_connection_str) as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT COUNT(*) FROM embeddings")
        count = cur.fetchone()[0]
        print(f"📊 Nombre total d'enregistrements: {count}\n")
        
        cur.execute("SELECT id, corpus, created_at FROM embeddings LIMIT 10")
        rows = cur.fetchall()
        
        print("📝 Aperçu des 10 premiers enregistrements:\n")
        for row in rows:
            print(f"ID: {row[0]}")
            print(f"Corpus: {row[1][:100]}..." if len(row[1]) > 100 else f"Corpus: {row[1]}")
            print(f"Créé le: {row[2]}")
            print("-" * 80)


📊 Nombre total d'enregistrements: 1062

📝 Aperçu des 10 premiers enregistrements:

ID: 1
Corpus: h: U B S bonjour
Créé le: 2025-12-03 11:06:12.020737
--------------------------------------------------------------------------------
ID: 2
Corpus: c: oui bonjour e j'appelle je sais pas si j'appelle au bon endroit e
Créé le: 2025-12-03 11:06:12.020737
--------------------------------------------------------------------------------
ID: 3
Corpus: h: je vous écoute
Créé le: 2025-12-03 11:06:12.020737
--------------------------------------------------------------------------------
ID: 4
Corpus: c: c'est pour
Créé le: 2025-12-03 11:06:12.020737
--------------------------------------------------------------------------------
ID: 9
Corpus: c: ce serait pour ma fille
Créé le: 2025-12-03 11:06:12.020737
--------------------------------------------------------------------------------
ID: 5
Corpus: c: e c'est pour savoir si la fac pendant l'été e a des professeurs ou des des gens qui font des stag...